In [2]:
%%capture

!pip install snowflake-connector-python
!pip install "snowflake-connector-python[secure-local-storage,pandas]"

In [3]:
%%capture

!pip install snowflake-sqlalchemy

In [4]:
%%capture

!pip install xgboost

In [5]:
! pip install pandas=="2.1.4"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 23.0 MB/s eta 0:00:0000:010:01
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.1
    Uninstalling pandas-2.2.1:
      Successfully uninstalled pandas-2.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autovizwidget 0.21.0 requires pandas<2.0.0,>=0.20.1, but you have pandas 2.1.4 which is incompatible.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.


In [1]:
import os
import pickle
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import metrics

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost
from datetime import datetime, timedelta
import time  
import pytz    
tz_NY = pytz.timezone('America/Indianapolis')

import snowflake_creds
import LOS_Preprocessing
from LOS_Preprocessing import preprocess_data

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.__version__

'2.1.4'

In [3]:
# Creating the connection engine 
engine = create_engine(URL(
         account = "kbxyguy-yh77168",
        #account="by44730.us-east-2.aws",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
    ))

In [4]:
query = """

WITH BASE AS (

    SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.PUBLIC.SIMULATION_DATA

),
    
BASE_WITH_FEATURES AS (

    SELECT *,
            MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
            DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
            CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
            CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
            CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
            DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
    FROM BASE 

)    

SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE-542
"""

In [5]:
# To test if the data is being inserted to the snowflake database
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test



def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas

    engine = create_engine(URL(
             account = "kbxyguy-yh77168",
        #account="by44730.us-east-2.aws",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
        ))  
    
    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [6]:
with engine.connect() as conn:
    
    # Loading the scoring data
    score_data = pd.DataFrame(pd.read_sql(query,con = conn))
    score_data.columns = [col.upper() for col in score_data.columns.tolist()]
    
    # Applying the preprocessing steps
    score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
    
    # Applying feature selection
    final_feats = pd.read_pickle('MODEL_FEATS.pkl')
    score_data_final = check_n_create_model_features(score_data_processed,final_feats)
    
    # Getting the predictions
    model = xgboost.XGBRegressor()
    model.load_model('MODEL_XGB.model')
    score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
    
    # Writing the dataframe to snowflake as a table
    score_data_final = score_data_final.reset_index()
    score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
    #status = insert_predictions_to_snowflake_table(score_data_table)

In [7]:
print(score_data.shape)
score_data.head()

(2362, 25)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT,AGE,ADMISSION_DEPOSIT,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS
0,2363,18,d,13,Y,4,radiotherapy,Q,B,2,127970,3,Trauma,Moderate,5,31-40,3778,2022-12-02,2023-01-21,Dec,Fri,Trauma-Moderate,Moderate-2,radiotherapy-Moderate,50
1,2364,28,b,11,X,3,anesthesia,R,F,3,74756,5,Trauma,Extreme,2,20-Nov,4710,2022-12-02,2022-12-22,Dec,Fri,Trauma-Extreme,Extreme-3,anesthesia-Extreme,20
2,2365,11,b,2,Y,4,gynecology,Q,D,4,53888,5,Emergency,Minor,2,41-50,7073,2022-12-02,2023-01-11,Dec,Fri,Emergency-Minor,Minor-4,gynecology-Minor,40
3,2366,23,a,6,X,4,gynecology,Q,F,1,107753,2,Trauma,Extreme,2,51-60,6203,2022-12-02,2023-01-01,Dec,Fri,Trauma-Extreme,Extreme-1,gynecology-Extreme,30
4,2367,2,c,5,Z,4,gynecology,S,F,2,107506,9,Emergency,Extreme,2,51-60,3583,2022-12-02,2022-12-22,Dec,Fri,Emergency-Extreme,Extreme-2,gynecology-Extreme,20


In [8]:
print(score_data_processed.shape)
score_data_processed.head()

(2362, 128)


,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,VISITORS_WITH_PATIENT,ADMISSION_DEPOSIT,LOS,HOSPITAL_TYPE_CODE_a,HOSPITAL_TYPE_CODE_b,HOSPITAL_TYPE_CODE_c,HOSPITAL_TYPE_CODE_d,HOSPITAL_TYPE_CODE_e,HOSPITAL_TYPE_CODE_f,HOSPITAL_TYPE_CODE_g,CITY_CODE_HOSPITAL_1,CITY_CODE_HOSPITAL_2,CITY_CODE_HOSPITAL_3,CITY_CODE_HOSPITAL_4,CITY_CODE_HOSPITAL_5,CITY_CODE_HOSPITAL_6,CITY_CODE_HOSPITAL_7,CITY_CODE_HOSPITAL_9,CITY_CODE_HOSPITAL_10,CITY_CODE_HOSPITAL_11,CITY_CODE_HOSPITAL_13,HOSPITAL_REGION_CODE_X,HOSPITAL_REGION_CODE_Y,HOSPITAL_REGION_CODE_Z,DEPARTMENT_TB & Chest disease,DEPARTMENT_anesthesia,DEPARTMENT_gynecology,DEPARTMENT_radiotherapy,DEPARTMENT_surgery,WARD_TYPE_P,WARD_TYPE_Q,WARD_TYPE_R,WARD_TYPE_S,WARD_TYPE_T,WARD_FACILITY_CODE_A,WARD_FACILITY_CODE_B,WARD_FACILITY_CODE_C,WARD_FACILITY_CODE_D,WARD_FACILITY_CODE_E,WARD_FACILITY_CODE_F,BED_GRADE_1,BED_GRADE_2,BED_GRADE_3,BED_GRADE_4,CITY_CODE_PATIENT_0,CITY_CODE_PATIENT_1,CITY_CODE_PATIENT_2,CITY_CODE_PATIENT_3,CITY_CODE_PATIENT_4,CITY_CODE_PATIENT_5,CITY_CODE_PATIENT_6,CITY_CODE_PATIENT_7,CITY_CODE_PATIENT_8,CITY_CODE_PATIENT_9,CITY_CODE_PATIENT_10,CITY_CODE_PATIENT_11,CITY_CODE_PATIENT_12,CITY_CODE_PATIENT_13,CITY_CODE_PATIENT_14,CITY_CODE_PATIENT_15,CITY_CODE_PATIENT_16,CITY_CODE_PATIENT_18,CITY_CODE_PATIENT_19,CITY_CODE_PATIENT_20,CITY_CODE_PATIENT_21,CITY_CODE_PATIENT_22,CITY_CODE_PATIENT_23,CITY_CODE_PATIENT_24,CITY_CODE_PATIENT_25,CITY_CODE_PATIENT_26,CITY_CODE_PATIENT_27,CITY_CODE_PATIENT_28,CITY_CODE_PATIENT_29,CITY_CODE_PATIENT_30,TYPE_OF_ADMISSION_Emergency,TYPE_OF_ADMISSION_Trauma,TYPE_OF_ADMISSION_Urgent,SEVERITY_OF_ILLNESS_Extreme,SEVERITY_OF_ILLNESS_Minor,SEVERITY_OF_ILLNESS_Moderate,AGE_0-10,AGE_20-Nov,AGE_21-30,AGE_31-40,AGE_41-50,AGE_51-60,AGE_61-70,AGE_71-80,AGE_81-90,AGE_91-100,ADMISSION_MONTH_Dec,ADMISSION_DAY_Fri,ADMISSION_ILLNESS_Emergency-Extreme,ADMISSION_ILLNESS_Emergency-Minor,ADMISSION_ILLNESS_Emergency-Moderate,ADMISSION_ILLNESS_Trauma-Extreme,ADMISSION_ILLNESS_Trauma-Minor,ADMISSION_ILLNESS_Trauma-Moderate,ADMISSION_ILLNESS_Urgent-Extreme,ADMISSION_ILLNESS_Urgent-Minor,ADMISSION_ILLNESS_Urgent-Moderate,ILLNESS_BEDGRADE_Extreme-1,ILLNESS_BEDGRADE_Extreme-2,ILLNESS_BEDGRADE_Extreme-3,ILLNESS_BEDGRADE_Extreme-4,ILLNESS_BEDGRADE_Minor-1,ILLNESS_BEDGRADE_Minor-2,ILLNESS_BEDGRADE_Minor-3,ILLNESS_BEDGRADE_Minor-4,ILLNESS_BEDGRADE_Moderate-1,ILLNESS_BEDGRADE_Moderate-2,ILLNESS_BEDGRADE_Moderate-3,ILLNESS_BEDGRADE_Moderate-4,DEPARTMENT_ILLNESS_TB & Chest disease-Extreme,DEPARTMENT_ILLNESS_TB & Chest disease-Minor,DEPARTMENT_ILLNESS_TB & Chest disease-Moderate,DEPARTMENT_ILLNESS_anesthesia-Extreme,DEPARTMENT_ILLNESS_anesthesia-Minor,DEPARTMENT_ILLNESS_anesthesia-Moderate,DEPARTMENT_ILLNESS_gynecology-Extreme,DEPARTMENT_ILLNESS_gynecology-Minor,DEPARTMENT_ILLNESS_gynecology-Moderate,DEPARTMENT_ILLNESS_radiotherapy-Extreme,DEPARTMENT_ILLNESS_radiotherapy-Minor,DEPARTMENT_ILLNESS_radiotherapy-Moderate,DEPARTMENT_ILLNESS_surgery-Extreme,DEPARTMENT_ILLNESS_surgery-Moderate
CASE_ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2363,4,5,3778,50,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False
2364,3,2,4710,20,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,False,Fal

In [9]:
print(score_data_final.shape)
score_data_final.head()

(2362, 23)


,CASE_ID,BED_GRADE_2,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,CITY_CODE_HOSPITAL_2,ADMISSION_MONTH_Nov,AGE_31-40,WARD_TYPE_S,ADMISSION_DEPOSIT,ILLNESS_BEDGRADE_Extreme-1,WARD_TYPE_P,AGE_71-80,VISITORS_WITH_PATIENT,AGE_51-60,SEVERITY_OF_ILLNESS_Minor,CITY_CODE_PATIENT_8,CITY_CODE_HOSPITAL_7,TYPE_OF_ADMISSION_Trauma,ADMISSION_MONTH_Oct,AGE_41-50,WARD_TYPE_Q,TYPE_OF_ADMISSION_Emergency,LOS,PREDICTED_LOS
0,2363,True,4,False,0,True,False,3778,False,False,False,5,False,False,False,False,True,0,False,True,False,50,58.0
1,2364,False,3,False,0,False,False,4710,False,False,False,2,False,False,False,False,True,0,False,False,False,20,28.0
2,2365,False,4,True,0,False,False,7073,False,False,False,2,False,True,False,False,False,0,True,True,True,40,31.0
3,2366,False,4,False,0,False,False,6203,True,False,False,2,True,False,False,False,True,0,False,True,False,30,35.0
4,2367,True,4,False,0,False,True,3583,False,False,False,2,True,False,False,False,False,0,False,False,True,20,28.0


In [10]:
print(score_data_table.shape)
score_data_table.head()

(2362, 47)


,CASE_ID,HOSPITAL_CODE,HOSPITAL_TYPE_CODE,CITY_CODE_HOSPITAL,HOSPITAL_REGION_CODE,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_x,DEPARTMENT,WARD_TYPE,WARD_FACILITY_CODE,BED_GRADE,PATIENTID,CITY_CODE_PATIENT,TYPE_OF_ADMISSION,SEVERITY_OF_ILLNESS,VISITORS_WITH_PATIENT_x,AGE,ADMISSION_DEPOSIT_x,ADMISSION_DATE,DISCHARGE_DATE,ADMISSION_MONTH,ADMISSION_DAY,ADMISSION_ILLNESS,ILLNESS_BEDGRADE,DEPARTMENT_ILLNESS,LOS_x,BED_GRADE_2,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_y,CITY_CODE_HOSPITAL_2,ADMISSION_MONTH_Nov,AGE_31-40,WARD_TYPE_S,ADMISSION_DEPOSIT_y,ILLNESS_BEDGRADE_Extreme-1,WARD_TYPE_P,AGE_71-80,VISITORS_WITH_PATIENT_y,AGE_51-60,SEVERITY_OF_ILLNESS_Minor,CITY_CODE_PATIENT_8,CITY_CODE_HOSPITAL_7,TYPE_OF_ADMISSION_Trauma,ADMISSION_MONTH_Oct,AGE_41-50,WARD_TYPE_Q,TYPE_OF_ADMISSION_Emergency,LOS_y,PREDICTED_LOS
0,2363,18,d,13,Y,4,radiotherapy,Q,B,2,127970,3,Trauma,Moderate,5,31-40,3778,2022-12-02,2023-01-21,Dec,Fri,Trauma-Moderate,Moderate-2,radiotherapy-Moderate,50,True,4,False,0,True,False,3778,False,False,False,5,False,False,False,False,True,0,False,True,False,50,58.0
1,2364,28,b,11,X,3,anesthesia,R,F,3,74756,5,Trauma,Extreme,2,20-Nov,4710,2022-12-02,2022-12-22,Dec,Fri,Trauma-Extreme,Extreme-3,anesthesia-Extreme,20,False,3,False,0,False,False,4710,False,False,False,2,False,False,False,False,True,0,False,False,False,20,28.0
2,2365,11,b,2,Y,4,gynecology,Q,D,4,53888,5,Emergency,Minor,2,41-50,7073,2022-12-02,2023-01-11,Dec,Fri,Emergency-Minor,Minor-4,gynecology-Minor,40,False,4,True,0,False,False,7073,False,False,False,2,False,True,False,False,False,0,True,True,True,40,31.0
3,2366,23,a,6,X,4,gynecology,Q,F,1,107753,2,Trauma,Extreme,2,51-60,6203,2022-12-02,2023-01-01,Dec,Fri,Trauma-Extreme,Extreme-1,gynecology-Extreme,30,False,4,False,0,False,False,6203,True,False,False,2,True,False,False,False,True,0,False,True,False,30,35.0
4,2367,2,c,5,Z,4,gynecology,S,F,2,107506,9,Emergency,Extreme,2,51-60,3583,2022-12-02,2022-12-22,Dec,Fri,Emergency-Extreme,Extreme-2,gynecology-Extreme,20,True,4,False,0,False,True,3583,False,False,False,2,True,False,False,False,False,0,False,False,True,20,28.0


# Scoring Function for Deployment:

In [15]:
query = """

WITH BASE AS (

    SELECT CASE_ID,
           COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
           COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
           COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
           COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
           COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
           COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
           COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
           COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
           COALESCE(BED_GRADE,0) AS BED_GRADE,
           PATIENTID,
           COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
           COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
           COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
           COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
           COALESCE(AGE,'None') AS AGE,
           COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
           ADMISSION_DATE,
           DISCHARGE_DATE

    FROM HEALTHDB.PUBLIC.SIMULATION_DATA

),
    
BASE_WITH_FEATURES AS (

    SELECT *,
            MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
            DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
            CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
            CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
            CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
            DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
    FROM BASE 

)    

SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE-543
"""

In [16]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test


def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas

    engine = create_engine(URL(
            account = "kbxyguy-yh77168",
        #account="by44730.us-east-2.aws",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
        ))  
    
    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_EMAIL_MONITORED'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [17]:
def send_status_mail(mail_string):
    import mail_creds
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    from email.mime.base import MIMEBase
    from email.mime.text import MIMEText
    from email.utils import formatdate
    from email import encoders
    
    subject = 'Patient LOS Prediction - STATUS MAIL'
    mail_content = mail_string

    username= mail_creds.MAIL_ID
    password= mail_creds.APP_PASSWORD
    send_from = mail_creds.MAIL_ID
    send_to = mail_creds.MAIL_ID
    Cc = ''
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Cc'] = Cc
    msg['Date'] = formatdate(localtime = True)
    msg['Subject'] = subject
    msg.attach(MIMEText(mail_content, 'plain'))
    smtp = smtplib.SMTP('smtp.gmail.com',587)
    smtp.ehlo()
    smtp.starttls()
    smtp.login(username,password)
    smtp.sendmail(send_from, send_to.split(',') + msg['Cc'].split(','), msg.as_string())
    smtp.quit()    

In [18]:
def LOS_MODEL_DEPLOYMENT():
    try:
        import pickle
        import pandas as pd
        import numpy as np
        import sqlalchemy
        import snowflake.connector
        from sqlalchemy import create_engine

        import xgboost as xgb
        from datetime import datetime, timedelta
        import time  
        import pytz    
        tz_NY = pytz.timezone('America/Indianapolis')

        import warnings
        warnings.filterwarnings('ignore')   
        
        import snowflake_creds
        import LOS_Preprocessing
        from LOS_Preprocessing import preprocess_data
        
        # Creating the connection engine
        engine = create_engine(URL(
                account = "kbxyguy-yh77168",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
            ))
        
        mail_list = []
        
        with engine.connect() as conn:
    
            # Loading the scoring data
            score_data = pd.DataFrame(pd.read_sql(query,conn))
            score_data.columns = [col.upper() for col in score_data.columns.tolist()]
            mail_list.append('STEP-1: Loading Data complete')

            # Applying the preprocessing steps
            score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
            mail_list.append('STEP-2: Applying the Preprocessing Steps complete')

            # Applying feature selection
            final_feats = pd.read_pickle('MODEL_FEATS.pkl')
            score_data_final = check_n_create_model_features(score_data_processed,final_feats)
            mail_list.append('STEP-3: Applying the Feature Selection Steps complete')

            # Getting the predictions
            model = xgboost.XGBRegressor()
            model.load_model('MODEL_XGB.model')
            score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
            mail_list.append('STEP-4: Getting the Predictions complete')

            # Writing the dataframe to snowflake as a table
            score_data_final = score_data_final.reset_index()
            score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
            status = insert_predictions_to_snowflake_table(score_data_table)
            mail_list.append('STEP-5: Writing the data to snowflake complete')
            
            # Creating the mail body
            mail_string = ",\n ".join(map(str, mail_list))
            send_status_mail(mail_string)
            print('Success')
    
    except:
        
        mail_content = 'Schedule is failed'
        send_status_mail(mail_content)
        print('Failed')
        

In [19]:
LOS_MODEL_DEPLOYMENT()

Success


In [20]:
# Scheduling the notebook

import time
from datetime import datetime    
import pytz    
tz_NY = pytz.timezone('America/Indianapolis')   
 

# scheduled hours in 24-hour format
hours_list = ["18:29"]
 
while True:
    tz_NY = pytz.timezone('America/Indianapolis')   
    now = datetime.now(tz_NY)  # gets current datetime

    hour = str(now.hour) # gets current hour
    minute = str(now.minute) # gets current minute
    current_time = f"{hour}:{minute}" # combines current hour and minute

    # checks if current time is in the hours list
    if current_time in hours_list:
        LOS_MODEL_DEPLOYMENT()
    
    time.sleep(60) # waits a minute until it repeats


Success


KeyboardInterrupt: 

In [21]:
print(datetime.now())

2024-05-27 22:30:52.203873


In [22]:
tz_NY = pytz.timezone('America/Indianapolis') 
print(datetime.now(tz_NY))

2024-05-27 18:31:05.653112-04:00


Till above, the implementation of storing the data to the Snowflake and setting up a notebook instance which runs on a specific time is successfull. However, for the simulation purpose we will try ti generate the prediction at one one for 30 days present in the data from "2022-12-1" to "2022-12-31" using a loop which runs once and we get the data stored in the snowflake.  

# Live Data Scoring Simulation:

In [32]:
engine = create_engine(URL(
                account = "kbxyguy-yh77168",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
            ))

In [43]:
def scoring_query(a):
    query_sim = f"""

    WITH BASE AS (

        SELECT CASE_ID,
               COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
               COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
               COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
               COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
               COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
               COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
               COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
               COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
               COALESCE(BED_GRADE,0) AS BED_GRADE,
               PATIENTID,
               COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
               COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
               COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
               COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
               COALESCE(AGE,'None') AS AGE,
               COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
               ADMISSION_DATE,
               DISCHARGE_DATE

        FROM HEALTHDB.PUBLIC.SIMULATION_DATA

    ),

    BASE_WITH_FEATURES AS (

        SELECT *,
                MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
                DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,
                CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
                CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
                CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
                DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
        FROM BASE 

    )    

    SELECT * FROM BASE_WITH_FEATURES WHERE ADMISSION_DATE = CURRENT_DATE+{a}-544
    """
    return query_sim

In [44]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

In [45]:
def insert_predictions_to_snowflake_table(data):
    import pandas
    import snowflake.connector
    from snowflake.connector.pandas_tools import pd_writer, write_pandas
    

    engine = create_engine(URL(
                account = "kbxyguy-yh77168",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
            ))

    # Creating the logging table if not exists already
    table = 'TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_EMAIL_MONITORED'
    
    # Inserting the data to snowflake logging table
    data.to_sql(table, engine, index=False, if_exists='append', method=pd_writer)
    return 'Success'

In [46]:
def send_status_mail(mail_string):
    import mail_creds
    import smtplib
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    from email.mime.multipart import MIMEMultipart
    from email.mime.base import MIMEBase
    from email.mime.text import MIMEText
    from email.utils import formatdate
    from email import encoders
    
    subject = 'Patient LOS Prediction - STATUS MAIL'
    mail_content = mail_string

    username= mail_creds.MAIL_ID
    password= mail_creds.APP_PASSWORD
    send_from = mail_creds.MAIL_ID
    send_to = mail_creds.MAIL_ID
    Cc = ''
    msg = MIMEMultipart()
    msg['From'] = send_from
    msg['To'] = send_to
    msg['Cc'] = Cc
    msg['Date'] = formatdate(localtime = True)
    msg['Subject'] = subject
    msg.attach(MIMEText(mail_content, 'plain'))
    smtp = smtplib.SMTP('smtp.gmail.com',587)
    smtp.ehlo()
    smtp.starttls()
    smtp.login(username,password)
    smtp.sendmail(send_from, send_to.split(',') + msg['Cc'].split(','), msg.as_string())
    smtp.quit()    

In [47]:
def LOS_MODEL_DEPLOYMENT_SIM(x):
    try:
        import pickle
        import pandas as pd
        import numpy as np
        import sqlalchemy
        import snowflake.connector
        from sqlalchemy import create_engine

        import xgboost as xgb
        from datetime import datetime, timedelta
        import time  
        import pytz    
        tz_NY = pytz.timezone('America/Indianapolis')

        import warnings
        warnings.filterwarnings('ignore')   
        
        import snowflake_creds
        import LOS_Preprocessing
        from LOS_Preprocessing import preprocess_data
        
        # Creating the connection engine (way 1)
        engine = create_engine(URL(
                account = "kbxyguy-yh77168",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="PUBLIC"
            ))
        
        mail_list = []
        
        with engine.connect() as conn:
    
            # Loading the scoring data
#             print(scoring_query(x))
            score_data = pd.DataFrame(pd.read_sql(scoring_query(x),conn))
            score_data.columns = [col.upper() for col in score_data.columns.tolist()]
            print(score_data['ADMISSION_DATE'].max())
            mail_list.append('STEP-1: Loading Data complete')

            # Applying the preprocessing steps
            score_data_processed = LOS_Preprocessing.preprocess_data(score_data)
            mail_list.append('STEP-2: Applying the Preprocessing Steps complete')

            # Applying feature selection
            final_feats = pd.read_pickle('MODEL_FEATS.pkl')
            score_data_final = check_n_create_model_features(score_data_processed,final_feats)
            mail_list.append('STEP-3: Applying the Feature Selection Steps complete')

            # Getting the predictions
            model = xgboost.XGBRegressor()
            model.load_model('MODEL_XGB.model')
            score_data_final['PREDICTED_LOS'] = np.ceil(model.predict(score_data_final.drop('LOS',axis=1)))
            mail_list.append('STEP-4: Getting the Predictions complete')

            # Writing the dataframe to snowflake as a table
            score_data_final = score_data_final.reset_index()
            score_data_table = pd.merge(score_data,score_data_final,on='CASE_ID',how='left')
            status = insert_predictions_to_snowflake_table(score_data_table)
            mail_list.append('STEP-5: Writing the data to snowflake complete')
            
            # Creating the mail body
            mail_string = ",\n ".join(map(str, mail_list))
            send_status_mail(mail_string)
            print('Success')
    
    except Exception as e:
        
        mail_content = 'Schedule is failed'
        send_status_mail(mail_content)
        print('Failed')
        print(e)
        

In [48]:
#LOS_MODEL_DEPLOYMENT_SIM(1)

In [49]:
# Scheduling the notebook (for live scoring simulation)

import time
from datetime import datetime    
import pytz    
tz_NY = pytz.timezone('America/Indianapolis')   
 

# scheduled hours in 24-hour format
#hours_list = ["10:10","17:17"]
 
for i in range(1,31):
#     tz_NY = pytz.timezone('Asia/Kolkata')   
#     now = datetime.now(tz_NY)  # gets current datetime

#     hour = str(now.hour) # gets current hour
#     minute = str(now.minute) # gets current minute
#     current_time = f"{hour}:{minute}" # combines current hour and minute

#     # checks if current time is in the hours list
#     if current_time in hours_list:
#         LOS_MODEL_DEPLOYMENT()
    
    # Run the scoring script
    LOS_MODEL_DEPLOYMENT_SIM(i)
    
    #time.sleep(60) # waits a minute until it repeats


2022-12-01
Success
2022-12-02
Success
2022-12-03
Success
2022-12-04
Success
2022-12-05
Success
2022-12-06
Success
2022-12-07
Success
2022-12-08
Success
2022-12-09
Success
2022-12-10
Success
2022-12-11
Success
2022-12-12
Success
2022-12-13
Success
2022-12-14
Success
2022-12-15
Success
2022-12-16
Success
2022-12-17
Success
2022-12-18
Success
2022-12-19
Success
2022-12-20
Success
2022-12-21
Success
2022-12-22
Success
2022-12-23
Success
2022-12-24
Success
2022-12-25
Success
2022-12-26
Success
2022-12-27
Success
2022-12-28
Success
2022-12-29
Success
2022-12-30
Success
